# Analyse des sentiments 


In [1]:
import pandas as pd
df = pd.read_csv('./data/data_JV_2020_2023_tokenised.csv')

In [2]:
df.head()

,Unnamed: 0,rating,title_x,text,parent_asin,title_y,main_category,text_tokenized,store,details,timestamp
0,0,4.0,It’s pretty sexual. Not my fav,I’m playing on ps5 and it’s interesting. It’s...,B07DK1H3H5,Cyberpunk 2077 - PC [Game Download Code in Box],Video Games,"['i', '’', 'm', 'playing', 'on', 'ps', '##5', ...",WARNER BROS,"{'Release date': 'December 10, 2020', 'Best Se...",2020-12-17 07:33:24.795
1,1,5.0,Good. A bit slow,Nostalgic fun. A bit slow. I hope they don’t...,B07SRWRH5D,Final Fantasy VII: Remake - PlayStation 4,Video Games,"['nos', '##tal', '##gic', 'fun', '.', 'a', 'bi...",Square Enix,"{'Release date': 'April 10, 2020', 'Best Selle...",2020-04-16 17:31:54.941
2,2,5.0,Will use again,Instant delivery!,B004RMK57U,Playstation Plus: 3 Month Membership [Digital ...,Video Games,"['instant', 'delivery', '!']",PlayStation,"{'Release date': 'March 22, 2011', 'Pricing': ...",2020-10-17 14:28:29.361
3,4,5.0,Price bumps it up from 4 stars,*it fits TWO wired Retro-bit 6 button controll...,B08L6782X9,Aenllosi Hard Carrying Case Compatible with Se...,Video Games,"['*', 'it', 'fits', 'two', 'wired', 'retro', '...",Aenllosi,{'Pricing': 'The strikethrough price is the Li...,2021-05-19 20:24:30.253
4,5,1.0,It's an Auto-renew scam,Sony and Amazon are collaborating in an Auto-r...,B017V6YVDC,PlayStation Plus: 1 Month Membership [Digital ...,Video Games,"['sony', 'and', 'amazon', 'are', 'collaboratin...",PlayStation,"{'Release date': 'November 11, 2015', 'Pricing...",2020-12-12 01:54:34.794


In [ ]:
df['title_y'] = df['title_y'].fillna('')
jeu = 'final fantasy'

nombre_obs = df[df['title_y'].str.contains(jeu, case=False)].shape[0]
print(f"Nombre d'observations contenant '{jeu}':", nombre_obs)



Nombre d'observations contenant 'final fantasy': 3664


In [4]:
from transformers import BertTokenizer
import numpy as np 

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")



def tokenize_comments(comment):
    if isinstance(comment, str):
        tokens = tokenizer.tokenize(comment)
        tokens = ['[CLS]'] + tokens + ['[SEP]']
        token_ids = tokenizer.convert_tokens_to_ids(tokens)
        return token_ids
    else:
        return np.nan

df['tokenized_comments'] = df['text'].apply(lambda x: tokenize_comments(x))

# Suppression des espaces

In [5]:
import ast

# on retire les '/' car ca rallonge le nombre de tokens
df['text_tokenized'] = df['text_tokenized'].apply(lambda x: ast.literal_eval(x))

In [6]:
df['text_tokenized'] = df['text_tokenized'].apply(lambda x : ['[CLS]'] + x + ['[SEP]'])

# Padding

In [7]:
df.loc[df['text_tokenized'].str.len()>300,:].shape

(12653, 11)

In [8]:
df_ = df.loc[df['text_tokenized'].str.len()<300,:].copy() # on perd 12501 observations ce qui n'est pas enorme

In [1]:
def padding(row):
    if len(row) == 300:
        pass
    else:
        row = row + ['[PAD]'] * (300 - len(row))
    return row

df_['text_tokenized'] = df_['text_tokenized'].apply(lambda x: padding(x))


NameError: name 'df_' is not defined

# Attention masks

In [10]:
def attention_mask(row):
    mask = [1 if token != '[PAD]' else 0 for token in row]
    return mask

df_['attention_mask'] = df_['text_tokenized'].apply(lambda x: attention_mask(x))

# Unique token ID

In [11]:
df_['token_ids'] = df_['text_tokenized'].apply(lambda x : tokenizer.convert_tokens_to_ids(x))

In [12]:
import torch
df_['token_ids_unsqueeze'] = df_['token_ids'].apply(lambda x : torch.tensor(x).unsqueeze(0))

In [13]:
df_['attention_mask_unsqueeze'] = df_['attention_mask'].apply(lambda x : torch.tensor(x).unsqueeze(0))

In [14]:
df_.head()

,Unnamed: 0,rating,title_x,text,parent_asin,title_y,main_category,text_tokenized,store,details,timestamp,attention_mask,token_ids,token_ids_unsqueeze,attention_mask_unsqueeze
0,0,4.0,It’s pretty sexual. Not my fav,I’m playing on ps5 and it’s interesting. It’s...,B07DK1H3H5,Cyberpunk 2077 - PC [Game Download Code in Box],Video Games,"[[CLS], i, ’, m, playing, on, ps, ##5, and, it...",WARNER BROS,"{'Release date': 'December 10, 2020', 'Best Se...",2020-12-17 07:33:24.795,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1045, 1521, 1049, 2652, 2006, 8827, 2629...","[[tensor(101), tensor(1045), tensor(1521), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
1,1,5.0,Good. A bit slow,Nostalgic fun. A bit slow. I hope they don’t...,B07SRWRH5D,Final Fantasy VII: Remake - PlayStation 4,Video Games,"[[CLS], nos, ##tal, ##gic, fun, ., a, bit, slo...",Square Enix,"{'Release date': 'April 10, 2020', 'Best Selle...",2020-04-16 17:31:54.941,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 16839, 9080, 12863, 4569, 1012, 1037, 29...","[[tensor(101), tensor(16839), tensor(9080), te...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
2,2,5.0,Will use again,Instant delivery!,B004RMK57U,Playstation Plus: 3 Month Membership [Digital ...,Video Games,"[[CLS], instant, delivery, !, [SEP], [PAD], [P...",PlayStation,"{'Release date': 'March 22, 2011', 'Pricing': ...",2020-10-17 14:28:29.361,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[101, 7107, 6959, 999, 102, 0, 0, 0, 0, 0, 0, ...","[[tensor(101), tensor(7107), tensor(6959), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
3,4,5.0,Price bumps it up from 4 stars,*it fits TWO wired Retro-bit 6 button controll...,B08L6782X9,Aenllosi Hard Carrying Case Compatible with Se...,Video Games,"[[CLS], *, it, fits, two, wired, retro, -, bit...",Aenllosi,{'Pricing': 'The strikethrough price is the Li...,2021-05-19 20:24:30.253,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 1008, 2009, 16142, 2048, 17502, 22307, 1...","[[tensor(101), tensor(1008), tensor(2009), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."
4,5,1.0,It's an Auto-renew scam,Sony and Amazon are collaborating in an Auto-r...,B017V6YVDC,PlayStation Plus: 1 Month Membership [Digital ...,Video Games,"[[CLS], sony, and, amazon, are, collaborating,...",PlayStation,"{'Release date': 'November 11, 2015', 'Pricing...",2020-12-12 01:54:34.794,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[101, 8412, 1998, 9733, 2024, 20295, 1999, 201...","[[tensor(101), tensor(8412), tensor(1998), ten...","[[tensor(1), tensor(1), tensor(1), tensor(1), ..."


# Getting the embedding

In [17]:
df_test = df_.iloc[:10,:].copy()
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 labels pour sentiment positif/négatif

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
token_ids_tensor = torch.stack(df_['token_ids_unsqueeze'].tolist())
attention_mask_tensor = torch.stack(df_['attention_mask_unsqueeze'].tolist())


In [26]:
token_ids_tensor.shape

torch.Size([611317, 1, 300])

In [23]:
# Mettez le modèle en mode évaluation
model.eval()

# Passez les données à travers le modèle
with torch.no_grad():
    outputs = model(input_ids=token_ids_tensor, attention_mask=attention_mask_tensor)

# Obtenez les prédictions
predictions = torch.argmax(outputs.logits, dim=1)


ValueError: too many values to unpack (expected 2)

In [ ]:
df_.to_csv('./data/df_embedding.csv')